In [1]:
import os
import json
from tqdm import tqdm
import shutil

In [4]:
# path 
pj_path = 'C:/Users/Parknamgyu/Documents/ds_study/DL_project'   # project folder path
img_path = pj_path + '/temp_datasets/temp_datasets/image'   # img path
fashion_path = pj_path +'/temp_datasets/temp_datasets/fa'   # json labeling path
convert_path = pj_path + '/temp_datasets/temp_datasets/yolo_category_labeling'    # txt labeling 저장 폴더 path
wrong_img_path = pj_path + '/temp_datasets/temp_datasets/wrong_datasets/train/wrong_img'    # 라벨링 잘못 저장된 이미지 이동할 폴더 path
wrong_json_path = pj_path + '/temp_datasets/temp_datasets/wrong_datasets/train/wrong_json' # 라벨링 잘못 저장된 json 파일 이동할 폴더 path

In [2]:
MATERIAL_CATEGORIES_Dict = ["패딩", "퍼", "무스탕", "스웨이드", "앙고라", "코듀로이", "시퀸/글리터", "데님", "저지", "트위드", "벨벳", "비닐/PVC", "울/캐시미어", "합성섬유", "헤어 니트", "니트", "레이스", "린넨", "메시", "플리스", "네오프렌", "실크", "스판덱스", "자카드", "가죽", "면", "시폰", "우븐"]

In [46]:
MATERIAL_CATEGORIES_Dict
dictionary = {i : MATERIAL_CATEGORIES_Dict[i] for i in range(len(MATERIAL_CATEGORIES_Dict))}
print(dictionary)

{0: '패딩', 1: '퍼', 2: '무스탕', 3: '스웨이드', 4: '앙고라', 5: '코듀로이', 6: '시퀸/글리터', 7: '데님', 8: '저지', 9: '트위드', 10: '벨벳', 11: '비닐/PVC', 12: '울/캐시미어', 13: '합성섬유', 14: '헤어 니트', 15: '니트', 16: '레이스', 17: '린넨', 18: '메시', 19: '플리스', 20: '네오프렌', 21: '실크', 22: '스판덱스', 23: '자카드', 24: '가죽', 25: '면', 26: '시폰', 27: '우븐'}


In [41]:
MATERIAL_CATEGORIES_Dict_C = {string : i for i,string in enumerate(MATERIAL_CATEGORIES_Dict)}
MATERIAL_CATEGORIES_Dict_C

{'패딩': 0,
 '퍼': 1,
 '무스탕': 2,
 '스웨이드': 3,
 '앙고라': 4,
 '코듀로이': 5,
 '시퀸/글리터': 6,
 '데님': 7,
 '저지': 8,
 '트위드': 9,
 '벨벳': 10,
 '비닐/PVC': 11,
 '울/캐시미어': 12,
 '합성섬유': 13,
 '헤어 니트': 14,
 '니트': 15,
 '레이스': 16,
 '린넨': 17,
 '메시': 18,
 '플리스': 19,
 '네오프렌': 20,
 '실크': 21,
 '스판덱스': 22,
 '자카드': 23,
 '가죽': 24,
 '면': 25,
 '시폰': 26,
 '우븐': 27}

In [45]:
MATERIAL_CATEGORIES_Dict_C

{'패딩': 0,
 '퍼': 1,
 '무스탕': 2,
 '스웨이드': 3,
 '앙고라': 4,
 '코듀로이': 5,
 '시퀸/글리터': 6,
 '데님': 7,
 '저지': 8,
 '트위드': 9,
 '벨벳': 10,
 '비닐/PVC': 11,
 '울/캐시미어': 12,
 '합성섬유': 13,
 '헤어 니트': 14,
 '니트': 15,
 '레이스': 16,
 '린넨': 17,
 '메시': 18,
 '플리스': 19,
 '네오프렌': 20,
 '실크': 21,
 '스판덱스': 22,
 '자카드': 23,
 '가죽': 24,
 '면': 25,
 '시폰': 26,
 '우븐': 27}

In [44]:
li = [f for f in os.listdir(fashion_path) if f != '.DS_Store'] 
img_name = [f for f in os.listdir(img_path) if f != '.DS_Store']

for label in tqdm(li):
    with open(os.path.join(fashion_path, label), "r") as f:
        fa = json.load(f)
    
    if label.split('.')[0] + '.jpg' not in img_name:
        continue

    filename = str(fa['이미지 정보']['이미지 식별자']) + '.jpg'
    ann = fa['데이터셋 정보']['데이터셋 상세설명']['렉트좌표']
    img_w = fa['이미지 정보']['이미지 너비']
    img_h = fa['이미지 정보']['이미지 높이']
    
    for key, value in ann.items():
        if len(value[0]) > 0:
            try:
                class_label = (fa['데이터셋 정보']['데이터셋 상세설명']['라벨링'][key][0]['소재'])
                bbox=[]
                class_label = class_label[0]
            except:
                # 라벨링이 잘못된 파일인 경우 이미지, json 파일 이동
                print(filename)
                shutil.move(os.path.join(img_path, filename), os.path.join(wrong_img_path, filename))
                shutil.move(os.path.join(fashion_path, filename.split('.')[0] + '.json'), os.path.join(wrong_json_path, filename.split('.')[0] + '.json'))
                break

            for i in value[0].values():
                bbox.append(float(i))
                
            x, y, w, h = list(map(float, bbox))
            
            # yolo 라벨 좌표형식으로 변환 (yolo 라벨은 x_center, y_center, w, h를 0~1 사이값으로 정규화하여 사용)
            x = (x + w/2) / img_w
            y = (y + h/2) / img_h
            w = w / img_w
            h = h / img_h

            # 클래스, 좌표 txt 파일로 저장
            with open(os.path.join(convert_path, filename.split('.')[0] + '.txt'), 'a') as t:
                try:
                    t.write(str(MATERIAL_CATEGORIES_Dict_C[class_label]) + ' ' + str(x) + ' ' + str(y) + ' ' + str(w) + ' ' + str(h) + '\n')
                except:
                    print(filename)
                    os.remove(os.path.join(convert_path, filename.split('.')[0] + '.txt'))
                    shutil.move(os.path.join(img_path, filename), os.path.join(wrong_img_path, filename))
                    shutil.move(os.path.join(fashion_path, filename.split('.')[0] + '.json'), os.path.join(wrong_json_path, filename.split('.')[0] + '.json'))
                    break

  0%|          | 373/120975 [00:00<01:04, 1872.95it/s]

1000406.jpg
1000466.jpg
1000476.jpg
1000486.jpg
1000546.jpg
1000556.jpg
1000566.jpg
1000576.jpg
1001986.jpg
1002746.jpg
1002846.jpg
1002856.jpg
1003525.jpg


  0%|          | 561/120975 [00:00<01:13, 1631.02it/s]

1003535.jpg
1003545.jpg
1003555.jpg
1003565.jpg
1003575.jpg
1003585.jpg
1003595.jpg
1003605.jpg
1003615.jpg
1003635.jpg
1003645.jpg
1003655.jpg
1003665.jpg
1003675.jpg
1003695.jpg
1003705.jpg
1003725.jpg
1003735.jpg
1003745.jpg
1003756.jpg
1003766.jpg
1003776.jpg
1003786.jpg
1003831.jpg
1003861.jpg
1003871.jpg
1003881.jpg
1003891.jpg
1003901.jpg
1003911.jpg
1003951.jpg
1003961.jpg
1003991.jpg
1004011.jpg
1004021.jpg
1004031.jpg
1004141.jpg
1004151.jpg
1004181.jpg
1004191.jpg
1004291.jpg
1004759.jpg
1004769.jpg
1004779.jpg
1004789.jpg
1004799.jpg
1004809.jpg
100482.jpg
100492.jpg
1004991.jpg
1005001.jpg
1005021.jpg
1005031.jpg
1005041.jpg
1005051.jpg
1005071.jpg
1005081.jpg
1005091.jpg
1005101.jpg
1005111.jpg
1005121.jpg
1005211.jpg
1005313.jpg
1005323.jpg
1005333.jpg
1005343.jpg
1005433.jpg
1005443.jpg
10055.jpg
1005653.jpg
1005663.jpg
1005693.jpg
1005733.jpg
1005743.jpg
1005753.jpg
1005813.jpg
1005823.jpg
1005833.jpg
1005843.jpg
1005853.jpg
1005863.jpg
1005873.jpg
1005883.jpg
1005893.

  1%|          | 887/120975 [00:00<01:25, 1398.43it/s]

...

996277.jpg
996297.jpg
996307.jpg
996647.jpg
996657.jpg
996667.jpg
996817.jpg
996837.jpg


100%|█████████▉| 120908/120975 [03:05<00:00, 420.02it/s]

998571.jpg
998581.jpg
998651.jpg
998661.jpg
998671.jpg
998891.jpg
998911.jpg
998931.jpg
999.jpg
999121.jpg
999131.jpg
999141.jpg
999151.jpg


100%|██████████| 120975/120975 [03:05<00:00, 651.82it/s]

9997.jpg


In [ ]:
li = [f for f in os.listdir(fashion_path) if f != '.DS_Store'] 
img_name = [f for f in os.listdir(img_path) if f != '.DS_Store']

for label in tqdm(li):
    with open(os.path.join(fashion_path, label), "r") as f:
        fa = json.load(f)
    
    if label.split('.')[0] + '.jpg' not in img_name:
        continue

    filename = str(fa['이미지 정보']['이미지 식별자']) + '.jpg'
    ann = fa['데이터셋 정보']['데이터셋 상세설명']['렉트좌표']
    img_w = fa['이미지 정보']['이미지 너비']
    img_h = fa['이미지 정보']['이미지 높이']
    
    for key, value in ann.items():
        if len(value[0]) > 0:
            try:
                class_label = (fa['데이터셋 정보']['데이터셋 상세설명']['라벨링'][key][0]['소재'])
                bbox=[]
                class_label = class_label[0]
            except:
                # 라벨링이 잘못된 파일인 경우 이미지, json 파일 이동
                print(filename)
                shutil.move(os.path.join(img_path, filename), os.path.join(wrong_img_path, filename))
                shutil.move(os.path.join(fashion_path, filename.split('.')[0] + '.json'), os.path.join(wrong_json_path, filename.split('.')[0] + '.json'))
                break

            for i in value[0].values():
                bbox.append(float(i))
                
            x, y, w, h = list(map(float, bbox))
            
            # yolo 라벨 좌표형식으로 변환 (yolo 라벨은 x_center, y_center, w, h를 0~1 사이값으로 정규화하여 사용)
            x = (x + w/2) / img_w
            y = (y + h/2) / img_h
            w = w / img_w
            h = h / img_h

            # 클래스, 좌표 txt 파일로 저장
            with open(os.path.join(convert_path, filename.split('.')[0] + '.txt'), 'a') as t:
                try:
                    t.write(str(MATERIAL_CATEGORIES_Dict_C[class_label]) + ' ' + str(x) + ' ' + str(y) + ' ' + str(w) + ' ' + str(h) + '\n')
                except:
                    print(filename)
                    os.remove(os.path.join(convert_path, filename.split('.')[0] + '.txt'))
                    shutil.move(os.path.join(img_path, filename), os.path.join(wrong_img_path, filename))
                    shutil.move(os.path.join(fashion_path, filename.split('.')[0] + '.json'), os.path.join(wrong_json_path, filename.split('.')[0] + '.json'))
                    break